In [ ]:

!pip install -q pandas openpyxl ipywidgets

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
from datetime import datetime
import os

# Configurações
REPO_RAW_BASE = "https://github.com/fabiaalves/conversor-planner-bahia/raw/main"
MAPEAMENTO_FILENAME = "Mapeamento_Colunas.xlsx"
MAPEAMENTO_URL = f"{REPO_RAW_BASE}/{MAPEAMENTO_FILENAME}"

# Diretório para backups
backup_dir = '/content/backups'
os.makedirs(backup_dir, exist_ok=True)

# Histórico simples
log_conversoes = []

def carregar_mapeamento():
    try:
        df_map = pd.read_excel(MAPEAMENTO_URL)
        return df_map
    except Exception as e:
        print("Erro ao carregar mapeamento:", e)
        return None

def salvar_backup(df_final, nome):
    caminho = f"{backup_dir}/{nome}"
    df_final.to_excel(caminho, index=False)
    backups = sorted(os.listdir(backup_dir), key=lambda x: os.path.getmtime(os.path.join(backup_dir, x)))
    if len(backups) > 5:
        os.remove(os.path.join(backup_dir, backups[0]))

def converter_planilha():
    clear_output(wait=True)
    
    uploader = widgets.FileUpload(accept='.xlsx', multiple=False)
    botao = widgets.Button(description='Converter para Plano de Ação')
    baixar_ultimo = widgets.Button(description='⬇️ Baixar Último Arquivo')
    status = widgets.Output()
    
    df_map = carregar_mapeamento()
    if df_map is None:
        print("Não foi possível carregar o arquivo de mapeamento. Verifique o arquivo no GitHub.")
        return
    
    def on_button_click(b):
        with status:
            status.clear_output()
            if len(uploader.value) == 0:
                print("⚠️ Nenhum arquivo enviado.")
                return
            for nome_arquivo, file_info in uploader.value.items():
                content = file_info['content']
                try:
                    df = pd.read_excel(pd.io.common.BytesIO(content))
                except Exception as e:
                    print("Erro ao ler arquivo enviado:", e)
                    return
                
                df_final = pd.DataFrame()
                for _, row in df_map.iterrows():
                    origem = row.get('Coluna do Planner (origem)')
                    destino = row.get('Coluna na nova planilha (destino)')
                    regra = str(row.get('Transformação / Regra especial', '')).strip().lower()
                    
                    if pd.isna(origem) or pd.isna(destino):
                        continue
                    
                    if regra == 'valor fixo':
                        df_final[destino] = [row['Formato esperado']] * len(df)
                    elif regra == 'data (dd/mm/yyyy)':
                        df_final[destino] = pd.to_datetime(df[origem], errors='coerce').dt.strftime('%d/%m/%Y')
                    else:
                        df_final[destino] = df.get(origem, '')
                
                hoje = datetime.now().strftime('%d-%m-%Y')
                nome_final = f"PLANO DE AÇÃO - PROJETO BAHIA QUE PRODUZ E ALIMENTA_{hoje}.xlsx"
                df_final.to_excel(nome_final, index=False)
                salvar_backup(df_final, nome_final)
                files.download(nome_final)
                
                log_conversoes.append(f"✔️ {nome_final} gerado em {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
                print(f"✔️ Arquivo '{nome_final}' gerado e pronto para download.")
    
    def baixar_ultimo_arquivo(b):
        hoje = datetime.now().strftime('%d-%m-%Y')
        nome_final = f"PLANO DE AÇÃO - PROJETO BAHIA QUE PRODUZ E ALIMENTA_{hoje}.xlsx"
        if os.path.exists(nome_final):
            files.download(nome_final)
        else:
            print("Nenhum arquivo gerado para baixar hoje.")
    
    botao.on_click(on_button_click)
    baixar_ultimo.on_click(baixar_ultimo_arquivo)
    
    display(widgets.HTML("<h3>Upload da planilha a ser convertida (.xlsx):</h3>"))
    display(uploader)
    display(botao)
    display(baixar_ultimo)
    display(status)
    display(widgets.HTML("<h4>Histórico de conversões:</h4>"))
    display(widgets.HTML("<pre>" + "\n".join(log_conversoes) + "</pre>"))

converter_planilha()
